### Benchmarking Database Search

In [ ]:
import sys

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
import time

import pandas as pd
import yaml

from PhotonicsAI.Photon import llm_api, semantic_similarity, utils

with open("../../Frontend/prompts.yaml") as file:
    prompts = yaml.safe_load(file)

db_docs = utils.search_directory_for_docstrings(directory="../../KnowledgeBase")
list_of_docs = [i["docstring"] for i in db_docs]
list_of_cnames = [i["class_name"] for i in db_docs]

d = {
    "list_of_docs": list_of_docs,
    "list_of_cnames": list_of_cnames,
}

llm_api_selection_ = [
    "fireworks: llama-v3-70b-instruct",
    # 'fireworks: llama-v3-8b-instruct',
    "openai: gpt-4o",
    "openai: gpt-3.5",
    #   'groq: llama3-70b',
    # 'groq: llama3-8b',
    "fireworks: qwen2-72b-instruct",
    #   'fireworks: mixtral-8x22b-instruct',
    "google: gemini-1.5-flash",
    #   'google: gemini-1.5-pro',
]

### TASK: Parse Input

In [ ]:
# prompt = 'A high speed modulator connected to a VOA'
prompt = "Five cascaded MZIs, each with two input and two output ports."


def is_valid_yaml_custom(yaml_string: str) -> bool:
    try:
        data = yaml.safe_load(yaml_string)
        [_] = list(data.values())
        return True
    except:
        return False


run_time = []
valid_yaml = []
N = 2
for llm_api_selection in llm_api_selection_:
    print(llm_api_selection)
    c = 0
    start_time = time.time()
    for _i in range(N):
        raw_component_list = llm_api.call_llm(
            prompt, prompts["classify"], llm_api_selection
        )
        c += is_valid_yaml_custom(raw_component_list)
    end_time = time.time()
    valid_yaml.append(c / N)
    run_time.append((end_time - start_time) / N)

print("----")
print("N=", N)
df = pd.DataFrame(
    {
        "Model": llm_api_selection_,
        "Valid YAML": valid_yaml,
        "Time per request (s)": run_time,
    }
)
print(df.to_string(index=False))

### TASK: Search Database

In [ ]:
d["prompt"] = "Five cascaded MZIs, each with two input and two output ports."
d["raw_component_list"] = "components:\n  C1: MZI with two input and two output ports"

N = 50

# Initialize a dictionary to hold the response data
results = {}

# Data collection and processing
for llm_api_selection in llm_api_selection_:
    d["llm_api_selection"] = llm_api_selection
    print(llm_api_selection)
    response_counts = {}

    for _ in range(N):
        try:
            db_component_list, modified_yaml_data_all = llm_api.yaml_components_search(
                d
            )
            response = yaml.safe_load(db_component_list)["C1"]["component"]
        except:
            response = "ERROR!"

        if response in response_counts:
            response_counts[response] += 1
        else:
            response_counts[response] = 1

    results[llm_api_selection] = response_counts

# Output results
print(f"\nResults from {N} calls:")
for opt, counts in results.items():
    print(f"Model: {opt}")
    most_common_response = max(counts, key=counts.get)
    print(f"{counts[most_common_response]} times: {most_common_response}")
    print()

### LLM retrieval: order matters!

In [ ]:
import random

random.seed(1)

indices = list(range(len(list_of_docs)))
random.shuffle(indices)

d = {
    "list_of_docs": [list_of_docs[i] for i in indices],
    "list_of_cnames": [list_of_cnames[i] for i in indices],
}


d["llm_api_selection"] = llm_api_selection_[0]
# d['prompt'] = 'Five cascaded MZIs, each with two input and two output ports.'
d["raw_component_list"] = "components:\n  C1: MZI with two input and two output ports"
# d['raw_component_list'] = llm_api.call_llm(d['prompt'], prompts['classify'], d['llm_api_selection'])

db_component_list, modified_yaml_data_all = llm_api.yaml_components_search(d)
print(d["raw_component_list"])
print()
print(d["llm_api_selection"])
print()
print(modified_yaml_data_all)
print()
print(d["list_of_cnames"])

consistency check with 50 random documents ordering

In [ ]:
import random
from collections import Counter

res = {}
for llm_api_selection in llm_api_selection_:
    res[llm_api_selection] = []
    print(llm_api_selection)
    for seed_ in range(50):
        random.seed(seed_)

        indices = list(range(len(list_of_docs)))
        random.shuffle(indices)

        d = {
            "list_of_docs": [list_of_docs[i] for i in indices],
            "list_of_cnames": [list_of_cnames[i] for i in indices],
        }
        d["llm_api_selection"] = llm_api_selection
        d["raw_component_list"] = (
            "components:\n  C1: MZI with two input and two output ports"
        )

        db_component_list, modified_yaml_data_all = llm_api.yaml_components_search(d)

        data = yaml.safe_load(db_component_list)
        ret_comp = list(data.values())[0]["component"]
        res[llm_api_selection].append(ret_comp)

print()
for key, value in res.items():
    freq = Counter(value)
    print()
    print(f"{key}:")
    for item, count in sorted(freq.items(), key=lambda x: x[1], reverse=True):
        print(f"    {item}: {count}")

### BM25

In [ ]:
# d['prompt'] = 'Five cascaded MZIs, each with two input and two output ports.'
d["prompt"] = "MZI with two input and two output ports"
# d['prompt'] = 'MZI'

ii, ss = semantic_similarity.bm25(d["prompt"], d["list_of_docs"])
# ii, ss = semantic_similarity.dragon(d['prompt'], d['list_of_docs'])
# ii, ss = semantic_similarity.st(d['prompt'], d['list_of_docs'])

sorted_docs = [d["list_of_cnames"][i] for i in ii]
sorted_scores = sorted(ss)[::-1]

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 2.2))
plt.xlabel(d["prompt"])
bars = plt.bar(range(len(sorted_docs)), sorted_scores)
plt.xticks([])

for bar, label in zip(bars, sorted_docs):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        sorted_scores[0] * 0.1,
        label,
        ha="center",
        va="bottom",
        rotation=90,
        fontsize=8,
        fontweight="bold",
    )

plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.tight_layout()